In [114]:

import sys
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt

import tools
import extract
import classifier
import networks

importlib.reload(tools)
importlib.reload(extract)
importlib.reload(classifier)
importlib.reload(networks)

from extract import initPhogs,initImgs
from classifier import runSVM,getAccuracy

# Define symbol names

allNames = extract.getAllNames()
letters = "abcdefghijklmnopqrstuvwxyz"
lowercase = list(letters)
uppercase = list(letters.upper())
integers = [str(i) for i in range(10)]


In [55]:
# Set input

numTrain = 5000
numTest = 100
num = numTrain+numTest

names = integers


In [56]:
# Get input

%time trainPhogs,trainLabels,testPhogs,testLabels = initPhogs(names,numTrain,numTest)
%time trainImgs,trainLabels,testImgs,testLabels = initImgs(names,numTrain,numTest)

print(trainPhogs.shape)
print(testPhogs.shape)
print(trainImgs.shape)
print(testImgs.shape)
print(trainLabels.shape)
print(testLabels.shape)

0 6914
1 26520
2 26141
3 10909
4 7396
5 3545
6 3118
7 2909
8 3068
9 3737
CPU times: user 38.7 s, sys: 13.1 s, total: 51.8 s
Wall time: 1min 52s
CPU times: user 25 s, sys: 6.41 s, total: 31.4 s
Wall time: 57.3 s
(41057, 1032)
(820, 1032)
(41057, 45, 45, 1)
(820, 45, 45, 1)
(41057,)
(820,)


In [106]:
# Neural network classifier

# tests simple neural network
def testSimple(trainPhogs,trainLabels,testPhogs,testLabels,num_classes,dropout_rate,batch_size,epochs):
    input_shape = trainPhogs[0].shape
    hidden_size = 32
                    
    model = networks.simple_model(input_shape,hidden_size,dropout_rate,num_classes)
    model = networks.train_model(model,trainPhogs,trainLabels,testPhogs,testLabels,epochs,batch_size)
    score = networks.test_model(model,testPhogs,testLabels,batch_size)

    print("\n\nLoss: "+str(np.around(score[0],2))+" Accuracy: "+str(np.around(score[1],2)))
    summ,weights = networks.get_info(model)
    
    return model,summ,weights

# tests 2d CNN
def testImgConv(trainImgs,trainLabels,testImgs,testLabels,num_classes,dropout_rate,batch_size,epochs,size):
    input_shape = trainImgs[0].shape
    
    model = networks.conv_img(input_shape,dropout_rate,num_classes,size=size)
    model = networks.train_model(model,trainImgs,trainLabels,testImgs,testLabels,epochs,batch_size)
    score = networks.test_model(model,testImgs,testLabels,batch_size)

    print("\n\nLoss: "+str(np.around(score[0],2))+" Accuracy: "+str(np.around(score[1],2)))
    summ,weights = networks.get_info(model)
    
    return model,summ,weights


dropout_rate = 0.2
batch_size = 32
epochs = 10

#%time simple,simple_sum,simple_weights = testSimple(trainPhogs,trainLabels,testPhogs,testLabels,len(names),dropout_rate,batch_size,epochs)
%time small,small_sum,small_weights = testImgConv(trainImgs,trainLabels,testImgs,testLabels,len(names),dropout_rate,batch_size,epochs,'small')
#%time big,big_sum,big_weights = testImgConv(trainImgs,trainLabels,testImgs,testLabels,len(names),dropout_rate,batch_size,epochs,'big')


(41057, 45, 45, 1)
(41057,)
(820, 45, 45, 1)
(820,)
Train on 41057 samples, validate on 820 samples
Epoch 1/10
41057/41057 [==============================] - 372s - loss: 14.6799 - acc: 0.0892 - val_loss: 14.6832 - val_acc: 0.0890
Epoch 2/10
41057/41057 [==============================] - 354s - loss: 14.6797 - acc: 0.0892 - val_loss: 14.6832 - val_acc: 0.0890
Epoch 3/10
41057/41057 [==============================] - 363s - loss: 14.6797 - acc: 0.0892 - val_loss: 14.6832 - val_acc: 0.0890
Epoch 4/10
34496/41057 [========================>.....] - ETA: 60s - loss: 14.6879 - acc: 0.0887

KeyboardInterrupt: 

In [ ]:
# SVM classifier

C = 0.7
gamma = "auto"

%prun results = runSVM(trainPhogs,trainLabels,testPhogs,C=C,gamma=gamma,style="ovr")
accuracy = getAccuracy(results,testLabels)
print(np.around(accuracy,2))

%prun results = runSVM(trainPhogs,trainLabels,testPhogs,C=C,gamma=gamma,style="ovo")
accuracy = getAccuracy(results,testLabels)
print(np.around(accuracy,2))
